In [1]:
import logging
import os
import time

import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import torch
import torch.nn as nn
import torchvision

import numpy as np
import nets
import train
from arg import *
from lib import *
from main import *
from nni.utils import merge_parameter
from sklearn import metrics
from sklearn.metrics import classification_report

In [2]:
def output_calc(output,label,alpha = 0.0,disp = False):
    outputn = output[:,1]-output[:,0]
    #outputn = -outputn #inverse
    outputn = outputn/max([max(outputn),-min(outputn)])
    outputn = outputn/2+0.5
    pred = torch.round(outputn-alpha)
    acc = torch.sum(pred == label.data).item()/len(label)
    confusion_marix = metrics.confusion_matrix(label.cpu(),pred.cpu(),labels = [0,1])
    confusion_marix = confusion_marix.transpose()
    #swap the pred and label for a trapsposed confusion matrix
#     confusion_marix = metrics.confusion_matrix(label.cpu(),pred.cpu(),labels = [0,1])
    if disp:
        print(acc)
        print(confusion_marix)
    #TP TN
    #PP PN
    return outputn,confusion_marix,acc

def draw_auc(label,score,fig = 0,name = "roc_auc"):
    plt.figure(fig)
    score = -score+1
    fpr, tpr, thresholds = metrics.roc_curve(label.cpu(), score.cpu(), pos_label=0)
    roc_auc= metrics.auc(fpr, tpr)
    print("roc_auc:{}".format(roc_auc))
    plt.plot(fpr,tpr,label='{}:{:4f}'.format(name,roc_auc),linewidth=1)
    plt.legend(loc="lower right")
    return fpr,tpr
#     for tpr_threshold in [0.8,0.9,0.99,0.9956,tpr[thresholds == thresholds[thresholds>0.5][-1]][0]]: #first threshoulds which >0.5
#         plt.plot(fpr[tpr>tpr_threshold][1],tpr_threshold,'o')
#         text = "fold {} TPR:{},FPR:{:.4f},Thresholds:{:.4f}".format(1,tpr_threshold,fpr[tpr>tpr_threshold][1],thresholds[tpr>tpr_threshold][1])
#         print(text)
    
def print4copy(input):
    for value in input:
        print(value)

def conf_calc(conf):
    TP = conf[0][0]
    TN = conf[1][1]
    FN = conf[1][0]
    FP = conf[0][1]
    TPR = TP / (TP + FN)
    SPC = TN / (FP + TN)
    F1S = 2*TP / (2*TP + FP + FN)
    return TPR,SPC,F1S

In [ ]:
log = False
TRIAL_NAME = "408 apex test" 
TRIAL_TIME = time.strftime("%y%m%d-%H%M", time.localtime())
TL_BASE = "state_dict/210129-1712_2.dict"
#TL_BASE = "state_dict/210106-1316_ap"
DATA_DIR = '/home/ray/TrainingData/317/ap'
fold_num = len(os.listdir(DATA_DIR))
args = vars(get_params())
args['TRIAL_TIME'] = TRIAL_TIME
args['TRIAL_NAME'] = TRIAL_NAME
args['fold_num'] = fold_num
args['flood_level'] = 0.0
args['epochs'] = 100
args['batch_size'] = 32

In [ ]:
dataloaders = lib.get_dataloaders(args, 1, 5, 2, DATA_DIR)
train_runner = train.TrainRunner(args, logger, dataloaders, 1)

In [ ]:
train_runner.load_state(TL_BASE)

此处往下是XL转PTB的5折检测

In [ ]:
output = dict()
label = dict()
bce_output = dict()
cm = numpy.empty([2,2])
for fold in range(1,6):
    dataloaders = lib.get_dataloaders(args, fold, 5, 2, DATA_DIR)
    train_runner = train.TrainRunner(args, logger, dataloaders, fold)
    output[fold] = train_runner.test()
    label[fold] = train_runner.val_label
    bce_output[fold],cm_temp = output_calc(output[fold],label[fold],alpha = 0.0)
    cm = cm+cm_temp

In [ ]:
sum_bce = sum_output = sum_label = torch.empty(0)
#[output,label,bce] = torch.load("save/314_5f.save")
for i in range(1,len(output)+1):
    sum_output = torch.cat([sum_output,output[i].cpu()])
    sum_label = torch.cat([sum_label,label[i].cpu()])
    sum_bce = torch.cat([sum_bce,bce_output[i].cpu()])

In [ ]:
plt.figure(0,[9,9])
fpr, tpr, thresholds = metrics.roc_curve(sum_label.cpu(), sum_bce.cpu(), pos_label=1)
roc_auc= metrics.auc(fpr, tpr)
fold_auc= roc_auc
plt.plot(fpr,tpr,label='{} , {:4f}'.format('all',roc_auc))
plt.legend(loc="lower right")
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
for tpr_threshold in [0.8,0.9,0.99,0.9956,tpr[thresholds == thresholds[thresholds>0.5][-1]]]: #first threshoulds 
    plt.plot(fpr[tpr>tpr_threshold][1],tpr_threshold,'o')
    text = "fold {} TPR:{},FPR:{:.4f},Thresholds:{:.4f}".format('all',tpr_threshold,fpr[tpr>tpr_threshold][1],thresholds[tpr>tpr_threshold][1])
    print(text)
plt.show()

普通N折检测,+3Plane

In [24]:
save_name = "421_LRCUT"
result = torch.load("save/"+save_name+".save")
trial_name = "PTB"

In [25]:
#read output
output = dict()
acc_plane = {0:[],1:[],2:[],3:[]}
sens_plane = {0:[],1:[],2:[],3:[]}
spec_plane = {0:[],1:[],2:[],3:[]}
conf_plane = {0:[],1:[],2:[],3:[]}
label = torch.empty(0)#label
output[2] = torch.empty(0)
output[0] = torch.empty(0)
output[3] = torch.empty(0)
output[1] = torch.empty(0)
fold_label = dict()
fold_output = dict()
final_output = dict()
bestB = [1,1,2]
keys = result[0][0].copy().keys()
for i in range(len(result)):
    fold_label[i] = result[i][1]
    label = torch.cat([label,result[i][1]])
    fold_output[i] = result[i][0]
    temp = torch.zeros(result[i][0][2].shape)
    for j in keys:
        temp += result[i][0][j]*bestB[j]
    fold_output[i][3] = temp/len(keys)
    for j in result[0][0].keys():
        output[j] = torch.cat([output[j] ,result[i][0][j]])
        _,conf,acc_temp = output_calc(fold_output[i][j] ,fold_label[i],0)
        sens,spec,f1s = conf_calc(conf)
        spec_plane[j].append(spec)
        sens_plane[j].append(sens)
        acc_plane[j].append(acc_temp)
        conf_plane[j].append(conf)

In [32]:
print4copy(acc_plane[2]+['\n']+sens_plane[2]+['\n']+spec_plane[2]+['\n'])
print4copy(acc_plane[3]+['\n']+sens_plane[3]+['\n']+spec_plane[3])

0.9704663762881122
0.9590811965811966


0.968560606060606
0.963159759121502


0.9712092130518234
0.9573198715006884


0.9726973334749814
0.9652777777777778


0.9609848484848484
0.9511158342189161


0.9772626605640041
0.9713936056294936


In [26]:
_,pred2 = torch.max(output[2], 1)
report = classification_report(label,pred2,target_names=["MI","H"],output_dict = True)
print(classification_report(label,pred2,target_names=["MI","H"],output_dict = False))

              precision    recall  f1-score   support

          MI       0.92      0.97      0.94      5463
           H       0.99      0.96      0.97     13310

    accuracy                           0.96     18773
   macro avg       0.95      0.97      0.96     18773
weighted avg       0.97      0.96      0.97     18773



In [27]:
%matplotlib widget
fig = plt.figure(0,[5,5])
ax1 = fig.add_subplot()
plt.title('ROC_AUC curve')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
minorLocator = MultipleLocator(0.01)
majorLocator = MultipleLocator(0.1)
# Set minor tick locations.
ax1.yaxis.set_minor_locator(minorLocator)
ax1.xaxis.set_minor_locator(minorLocator)
# ax1.yaxis.set_major_locator(majorLocator)
# ax1.xaxis.set_major_locator(majorLocator)
# Set grid to use minor tick locations. 
ax1.grid(which = 'minor',alpha=0.3)
ax1.grid(which = 'major',alpha=0.6)
ax1.axis([-0.05,0.5,0.5,1.05])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-0.05, 0.5, 0.5, 1.05)

In [29]:
#Draw area addon
x = np.linspace(0,1,5000)
tpr1_p = np.interp(x,fpr1,tpr1)
tpr2_p = np.interp(x,fpr2,tpr2)
plt.fill_between(x, tpr1_p, tpr2_p, where=tpr2_p >= tpr1_p,
                 facecolor='green', interpolate=True)
plt.fill_between(x, tpr1_p, tpr2_p, where=tpr2_p <= tpr1_p,
                 facecolor='red', interpolate=True)

In [28]:
#only plane2
bce_output,confusion_marix,acc= output_calc(output[2] ,label.cpu(),0,True)
#bce_output= output_calc(output[2],label[1].cpu(),0)
fpr1,tpr1 = draw_auc(label,bce_output,0,trial_name+" M1")

0.9647898577744634
[[ 5276   474]
 [  187 12836]]
roc_auc:0.9932115620237667


In [31]:
#ijk calc
accmax = 0
for i in range(60):
    if not i%10:
        print(i)
    for j in range(60):
        for k in range(60):
            i = i/1
            j = j/1
            k = k/1
            temp = output[0]*i+output[1]*j+output[2]*k
            temp/=(i+j+k)
            _,pred = torch.max(temp,1)
            acc = torch.sum(pred == label.data).item()/len(label)
            if accmax < acc:
                best = [i,j,k]
                accmax = acc
print(best)
print(accmax)

0
10
20
30
40
50
[9.0, 13.0, 31.0]
0.9691578330581154


In [30]:
output_final = torch.zeros((output[2]).shape)
for i,value in enumerate(bestB):
    output_final += output[i]*value
output_final/=sum(bestB)
_,pred = torch.max(output_final,1)
acc = torch.sum(pred == label.data).item()/len(label)
bce_output,confusion_marix,acc = output_calc(output_final ,label,-0.000,True)
fpr2,tpr2 = draw_auc(label,bce_output,0,trial_name+" M2")

0.9689980290843232
[[ 5222   341]
 [  241 12969]]
roc_auc:0.9935295058499546
